# Working with Namespaces in Pinecone

Namespaces are a unique concept in Pinecone -- they are logical separators of vectors within a single index. Namespaces are invaluable when tackling problems surrounding multitenancy and scaling in a financially sustainable manner. Stay tuned for an indepth writeup about namespaces that will touch on these issues and much more in the future.

In this notebook, we will showcase namespaces' utility by building a search application that deals with a simple multitenant situation: we will have 3 users ("tenants") who'd like results back in their native languages -- one user speaks English, another user speaks Italian, and another user speaks French. The language our Pinecone engine receives at query-time will determine where (i.e. which namespace) the query is routed.

Let's get started!

## Step 1. Setup

#### Our environment:

In [12]:
# Uncomment & run this cell if your environment does not have these libraries

# !pip3 install -qU \
#     langchain \
#     tiktoken \
#     datasets \
#     pinecone-client

# !pip3 install protobuf==3.20.3
# !pip3 install apache-beam==2.50.0

In [31]:
# Import everything we need 

from tqdm.auto import tqdm
from uuid import uuid4
import pinecone
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from datasets import load_dataset
from getpass import getpass
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
import pandas as pd

#### Demo context:

For this demo, we will be creating a mini search application that searches an index for content, based on a user's (or "tenant's") preferred language.

We will be working with three Wikipedia datasets [from HuggingFace](https://huggingface.co/datasets/wikipedia); one in English, one in French, and one in Italian. Each dataset contains `id`, `url`, `title`, and `text` columns. We will direct users' (i.e. tenants') queries to the appropriate namespace based on the language passed in at query-time.

## Step 2. Initialize Our Index

We will configure our Pinecone index to store `1536`-dimension vectors and to use cosine similarity as its similarity metric (see [here](https://docs.pinecone.io/docs/choosing-index-type-and-size) for more about dimensions & [here](https://www.pinecone.io/learn/vector-similarity/) for more about similarity metrics). 

We will be using OpenAI's [text-embedding-ada-002](https://openai.com/blog/new-and-improved-embedding-model) model to produce vector representations of our textual data, and since that model outputs `1536`-dimension vectors, that's what our index will have to be configured to intake.

To keep things straightforward, we will name our index `"namespaces-demo"`.


In [22]:
# Note: This step takes a little while to complete, so don't worry if it takes 30s or so!

# Find API key in console at app.pinecone.io
YOUR_API_KEY = getpass("Pinecone API Key: ")

# Find ENV (cloud region) next to API key in console
YOUR_ENV = input("Pinecone environment: ")

INDEX_NAME = 'namespaces-demo'

# Initialize Pinecone client
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)

# Create index
pinecone.create_index(
    name=INDEX_NAME,
    metric='cosine',
    dimension=1536)


Pinecone API Key:  ········
Pinecone environment:  us-east-1-aws


In [23]:
# Confirm we indeed created our "namespaces-demo" index

pinecone.list_indexes()

['namespaces-demo']


Now, to connect to the index we just created, we will use the [GRPCIndex class](https://github.com/pinecone-io/pinecone-python-client/blob/main/pinecone/core/grpc/index_grpc.py#L293) (this class is generally quicker than the [REST Index class](https://github.com/pinecone-io/pinecone-python-client/blob/main/pinecone/index.py#L49) when you've got a lot of data to process). 

You can see below with the `.describe_index_stats()` method that our index currently has `0` vectors in it (`'total_vector_count': 0`).

In [24]:
demo_index = pinecone.GRPCIndex(INDEX_NAME)

demo_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Step 3. Prepare Our Data

In our for our data to be indexed into our Pinecone index, we need to do a few things --

Besides loading up our data into memory, we also need to chunk up our text into small pieces, extract whatever metadata we are interested in, and vectorize those small chunks of text data. 

### Step 3a. Load & Preview Our Data

The data we'll be using is Wikipedia data in 3 different languages: English, French, and Italian. The datasets are from [HuggingFace](https://huggingface.co/datasets/wikipedia). Since they take a pretty long time to load in their entirety, we'll be using snippets.

If you want to load the datasets in their entirety, you can execute this code

```
from datasets import load_dataset

wiki_en = load_dataset("wikipedia", "20220301.en")
wiki_it = load_dataset("wikipedia", "20220301.it")
wiki_fr = load_dataset("wikipedia", "20220301.fr")
```

To preview the datasets, you'll want to drill down to the `train` section, like so (you can read more about the various methods available to run on HuggingFace's `DatasetDict` objects [here](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.DatasetDict)): 

```
wiki_en['train'][:1000]
```

In [25]:
# Load up our English, Italian, and French datasets (just the first 10 rows from each HuggingFace dataset)

wiki_en = load_dataset("wikipedia", "20220301.en", split="train[:10]")
wiki_it = load_dataset("wikipedia", "20220301.it", split="train[:10]")
wiki_fr = load_dataset("wikipedia", "20220301.fr", split="train[:10]")

In [26]:
# Preview data in a Pandas dataframe

# English
wiki_en.to_pandas().head(2)

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,25,https://en.wikipedia.org/wiki/Autism,Autism,Autism is a neurodevelopmental disorder charac...


In [27]:
# Italian
wiki_it.to_pandas().head(2)

,id,url,title,text
0,2,https://it.wikipedia.org/wiki/Organo%20a%20pompa,Organo a pompa,Lorgano a pompa è un tipo di organo a serbatoi...
1,3,https://it.wikipedia.org/wiki/Antropologia,Antropologia,Lantropologia (dal greco ἄνθρωπος ànthropos «u...


In [28]:
# French
wiki_fr.to_pandas().head(2)

,id,url,title,text
0,3,https://fr.wikipedia.org/wiki/Antoine%20Meillet,Antoine Meillet,"Paul Jules Antoine Meillet, né le à Moulins (..."
1,7,https://fr.wikipedia.org/wiki/Alg%C3%A8bre%20l...,Algèbre linéaire,L’algèbre linéaire est la branche des mathémat...


### Step 3b. Write Helper Functions to Chunk, Extract Metadata, and Create Embeddings

As noted before, in order to get our data into our Pinecone index, we'll need to chunk it all up, figure out what (if any) metadata we want to include, and then transform our chunks of text into embeddings.

You can read more about chunking [here](https://www.pinecone.io/learn/chunking-strategies/) and metadata [here](https://docs.pinecone.io/docs/metadata-filtering).

The steps we need to take are: 
- Initialize our embedding model
- Write a function (`tiktoken_len`) that will tokenize our text data
- Write a function that will determine the ideal chunk size for our text data, given the number of tokens in our data, and chunk our data up (`chunk_by_size`)
- Write a function that will sew everything together (`create_chunks_metadata_embeddings`) -- chunking our data, grabbing metadata from our data that we want in our index, and creating embeddings of our chunked-up text data


**Note:** You will need an [OpenAI](https://openai.com/) API Key for the next part, since we will be making API calls to an OpenAI-hosted model: `ada-002`

In [29]:
# Initialize our OpenAI model

OPENAI_API_KEY = getpass("OpenAI API Key: ")
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

OpenAI API Key:  ········


Chunking can be a complex process, but we are going to keep it simple by using Langchain's [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter). Into that method, we will insert a custom length function (`tiktoken_len` below). This will essentially let us recursively split our text data into `n` tokens, using the `tiktoken` [library](https://github.com/openai/tiktoken).

(If you don't know what "recursively" means, check out [this article](https://www.geeksforgeeks.org/introduction-to-recursion-data-structure-and-algorithm-tutorials/)!)

In [35]:
# Tell tiktoken what model we'd like to use for embeddings
tiktoken.encoding_for_model('text-embedding-ada-002')

# Intialize a tiktoken tokenizer (i.e. a tool that identifies individual tokens (words))
tokenizer = tiktoken.get_encoding('cl100k_base')

# Create our custom tiktoken function
def tiktoken_len(text: str) -> int:
    """
    Split up a body of text using a custom tokenizer.

    :param text: Text we'd like to tokenize.
    """
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

Now that we have our custom tiktoken function, we need to write an additional function that will come up with the ideal size of our chunks & split our text up according to that size:

In [34]:
def chunk_by_size(text: str, size: int = 50) -> list[Document]:
    """
    Chunk up text recursively.
    
    :param text: Text to be chunked up
    :return: List of Document items (i.e. chunks).|
    """
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = size,
    chunk_overlap = 20,
    length_function = tiktoken_len,
    add_start_index = True,
)
    return text_splitter.create_documents([text])

Now, we can write a final function to put everything together nicely:

In [32]:
def create_chunks_metadata_embeddings(dataset: pd.DataFrame) -> list[dict]:
    """
    Given a dataset, split text data into chunks, extract metadata, create embeddings for each chunk.

    :param dataset: Data we want to process.
    :return: List of data objects to upsert into our Pinecone index.
    """
    data_objs = []

    # For each row in our dataset:
    for index, row in tqdm(dataset.iterrows()):  # (tqdm library prints status of for-loop to console)        
        # Create chunks
        chunked_text = chunk_by_size(row["text"])
        
        # Extract just the string content from the chunk
        chunked_text = [c.page_content for c in chunked_text]

        # Extract some metadata, create an ID, and generate an embedding for the chunk. 
        # Wrap that all in a dictionary, and append that dictionary to a list (`data_objs`).
        for idx, text in enumerate(chunked_text):
            payload = {
                "metadata": {
                    "url": row["url"],
                    "title": row["title"],
                    "chunk_num": idx, 
                    "text_content": text  # there are 248 chars in this chunk of text 
                },
             "id": str(uuid4()),
            "values": embed.embed_documents([text])[0]  # --> list of len 248, each item of those 248 has a len of 1536
            }
            data_objs.append(payload)
            
    # Return list of dictionaries, each containing our metadata, ID, and embedding, per chunk.
    return data_objs
            
            


### Step 3b. Create Our Data Objects For Upsert

Now that we have written our helper functions, let's use them!

We will extract 3 rows from our datasets to upsert into our index.

(Don't worry if this part takes a few minutes to run -- Wikipedia articles (even just 3) are *long* and creating embeddings is a lot of work!)

#### English

In [36]:
# English
data_objs_en = create_chunks_metadata_embeddings(wiki_en.to_pandas().head(3))  # Grab the first 3 rows (Wikipedia articles) from dataset

3it [02:00, 40.21s/it]


In [37]:
# Our 3 rows of Wikipedia data have resulted in 737 data objects, each with metadata, an ID, and an embedding!
len(data_objs_en)

737

In [38]:
# Inspect one of our data objects
data_objs_en[0]

{'metadata': {'url': 'https://en.wikipedia.org/wiki/Anarchism',
  'title': 'Anarchism',
  'chunk_num': 0,
  'text_content': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As'},
 'id': '05164b5a-cffb-4dde-af71-1f90de58b80d',
 'values': [-0.01077616963985924,
  -0.004095339232160969,
  -0.012736669790924379,
  -0.01574978611015656,
  -0.026657533003867324,
  0.014789273088363006,
  -0.03752580672142263,
  -0.016012940617859607,
  -0.0004983484997166827,
  -0.0054176897020462135,
  0.00962486959998098,
  -0.017183978177138178,
  -0.00227464014614372,
  0.007460425171107266,
  0.001337975268780113,
  -0.00621044219084036,
  0.04736777785893605,
  0.011151164813336086,
  -0.021447077476637263,
  0.01931552782688772,
  -0.007690685365347434,
  0.01877606108609647,
  -0.007473582896492419,
  -0.0

#### Italian

In [39]:
# Italian:
data_objs_it = create_chunks_metadata_embeddings(wiki_it.to_pandas().head(3))

3it [01:20, 26.73s/it]


In [40]:
# See how many data objects we have created
len(data_objs_it)

490

In [41]:
# Preview
data_objs_it[0]

{'metadata': {'url': 'https://it.wikipedia.org/wiki/Organo%20a%20pompa',
  'title': 'Organo a pompa',
  'chunk_num': 0,
  'text_content': "Lorgano a pompa è un tipo di organo a serbatoio d'aria costituito da una (o più) tastiera, manuale, e da due pedali per azionare i mantici per"},
 'id': 'f5233507-9bee-4645-8bd8-087603f51917',
 'values': [-0.010293664656242946,
  0.00040683322974453286,
  0.016851781338083925,
  -0.023751395389730576,
  -0.01913771451464865,
  0.03598949771537777,
  -0.03058131484616604,
  -0.016168790323762977,
  -0.023458684422837255,
  -0.00560680650509899,
  0.014328893615852909,
  0.007324740465357274,
  -0.015123393747337522,
  0.009715212740770985,
  0.027040906895477886,
  -0.008411952495057804,
  0.005380304192936921,
  0.009262208116446846,
  -0.005387273745590299,
  -0.020657023907697077,
  0.009324931762020757,
  0.00910191352769343,
  -0.003707670695111051,
  -0.008795264386815951,
  -0.00742231094287548,
  -0.015360350155649011,
  -0.009652489095197073

#### French

In [42]:
# French:
data_objs_fr = create_chunks_metadata_embeddings(wiki_fr.to_pandas().head(3))

3it [00:30, 10.01s/it]


In [43]:
# See how many data objects we have created
len(data_objs_fr)

199

In [44]:
# Preview
data_objs_fr[0]

{'metadata': {'url': 'https://fr.wikipedia.org/wiki/Antoine%20Meillet',
  'title': 'Antoine Meillet',
  'chunk_num': 0,
  'text_content': 'Paul Jules Antoine Meillet, né le  à Moulins (Allier) et mort le  à Châteaumeillant (Cher), est le principal linguiste français des premières décennies du . Il est aussi'},
 'id': '825c8146-e1fb-45c8-a7f0-c548d350a3aa',
 'values': [-0.03204154628603385,
  0.003798774482537566,
  -0.0029181645085517146,
  -0.027806628780716693,
  -0.00423491750531716,
  0.04970035380447564,
  -0.006811825162762889,
  0.017352508559024555,
  0.007024902922456368,
  -0.04266879378818763,
  0.009035823028098855,
  0.0330802987929327,
  -0.003340990549700266,
  0.004201624119916969,
  0.006059394745850837,
  -0.021640695853511055,
  0.025609266275626687,
  0.037022237114431415,
  0.01756558585305674,
  0.005496736672286001,
  -0.020908242306029442,
  0.007890530942861327,
  0.008596350527080854,
  -0.008483152923587946,
  0.008110267193370328,
  0.0011777532174934394,
  

## Step 4. Upsert Our Data Objects into Our Index, into Different Namespaces

Now that we have created our data objects, it's time to index ("upsert") them into our index!

Since we want to sub-divide our index into language-based `namespaces`, you'll see below that each time we call the `.upsert()` method, we also specify a `namespace` parameter. For our example, each namespace corresponds to the language of the dataset. 

We will index our data into our index in small batches of `100`. Read more about batch-indexing [here](#https://docs.pinecone.io/docs/insert-data#batching-upserts).

In [46]:
BATCH_SIZE = 100

def batch_upsert(data: list[dict], index: pinecone.GRPCIndex, namespace: str):
    """
    Upsert data objects to a Pinecone index in batches.

    :param data: Data objects we want to upsert.
    :param index: Index into which we want to upsert our data objects.
    :namespace: Namespace within our index into which we want to upsert our data objects.
    """
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE]
        # print(batch)
        index.upsert(vectors=batch, namespace=namespace)


# NOTE:
# - In Production, you'll want to have a try/except loop here to catch upsert errors.
# - You'll also likely want to optimize your batching as you scale your data. Parallelization and using generator objects will 
#   significantly improve your batch performance.
# - Last, you'll want to confirm that the # of vectors you upsert matches the # of vectors you intend to upsert.

#### English

In [48]:
# English
batch_upsert(data_objs_en, demo_index, 'en')

In [49]:
# Confirm there are indeed 737 vectors added to our index in the 'en' namespace. Perfect!
demo_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'en': {'vector_count': 737}},
 'total_vector_count': 737}

#### Italian

In [50]:
# Italian
batch_upsert(data_objs_it, demo_index, 'it')

In [51]:
# Confirm there are 490 vectors added to our index in the 'it' namespace
demo_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'en': {'vector_count': 737}, 'it': {'vector_count': 490}},
 'total_vector_count': 1227}

#### French

In [52]:
# French
batch_upsert(data_objs_fr, demo_index, 'fr')

In [53]:
# Confirm there are 199 vectors added to our index in the 'fr' namespace. Perfect!
demo_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'en': {'vector_count': 737},
                'fr': {'vector_count': 199},
                'it': {'vector_count': 490}},
 'total_vector_count': 1426}

## Step 5. Query Our Data

Now that we have all of our data objects in our index, it's time for the fun part: querying.

First let's come up with some fun queries in different languages, then let's create tenants to ask those queries.

In [54]:
# English
query_en = "Who is Wilhelm Weitling?"  # This should get us back some chunks of text from our Anarchism Wikipedia article

# Italian
query_it = "Chi è un famoso antropologo?"  # Here we are asking "Who is a famous Anthropologist?" This should get us results from the Antropologia article

# French
query_fr = "Qu’est-ce que l’espace vectoriel?" # This one is very apropos: "What is vector space?" This should get us results from the Algèbre linéaire article

In [55]:
# Tenants dict
tenants = [{
            'name': 'Audrey',
            'native_language': 'en',
            'query': query_en
    
            },
           {
             "name": "Michele",
             "native_language": 'it',
             "query": query_it
           },
           {
             "name": 'Pierre',
             "native_language": 'fr',
             "query": query_fr
           }]


### Step 5a. Create Vectorized Queries to Send to our Index

Since we are dealing with vector space, we can't simply execute a natural language query. 

Instead, we need to turn our query into a language that's understandable by our vector database -- we need to *vectorize* our query. Once vectorized, *then* can send it on through to our index & get results back.


To vectorize our queries, we need to send them through the same model (`ada-002`) that we used to vectorize our Wikipedia articles.

In [56]:
def vectorize_query(model: OpenAIEmbeddings, query: str) -> list[float]:
    """
    Given a vectorization model & query, create an embedding.

    :param model: Model for creating embeddings.
    :param query: Query we want to vectorize/embed.
    :return: Vector/embedding.
    """
    return model.embed_query(query)
        

In [57]:
# Let's create our vectors for each of our 3 queries:
query_vector_en = vectorize_query(embed, query_en)
query_vector_it = vectorize_query(embed, query_it)
query_vector_fr = vectorize_query(embed, query_fr)

In [58]:
# Preview
query_vector_en

[0.007369487998005005,
 -0.010546107756370203,
 -0.0002366059920899268,
 0.00035501414027413,
 -0.023515060499212228,
 0.019813490993778136,
 -0.02638209419886517,
 0.015533131517677918,
 0.021038373892159318,
 -0.022047894033506436,
 0.021940210792344258,
 0.01634074558184595,
 0.006487841705537974,
 -0.0061345100285955115,
 0.009038559143906022,
 -0.0023370361842392482,
 0.03112010279329178,
 -0.019261620902079984,
 -0.008863575739662628,
 -0.012948762600785403,
 -0.030124043988412504,
 0.003073985216870392,
 -0.004825500296548307,
 0.014119804809908069,
 -0.005700416852103989,
 0.0030891278234128584,
 0.0174714065755328,
 -0.019086638429159163,
 -0.0024228454521331073,
 -0.023124713406612197,
 -0.008325166053109748,
 0.008883766813041823,
 -0.0019601492639538458,
 0.006040288123901178,
 -0.006824348108879489,
 -0.027485834382261473,
 0.01216806841558534,
 -0.007080092547010656,
 0.006878188798138005,
 0.0064373654190738455,
 0.006511397181711557,
 0.02321893531130653,
 0.01877705190

In [59]:
# Confirm that our query vectors are of the same dimension as our indexed vectors: 1536. Perfect!
len(query_vector_en)

1536

To save us some time, let's add these vectorized queries into our tenants dictionary:

In [60]:
# Define a list of new key-value pairs
new_key_value_pairs = [
    {'vector_query': query_vector_en},
    {'vector_query': query_vector_it},
    {'vector_query': query_vector_fr}
]

# Loop through the list of dictionaries and the list of new key-value pairs
for tenant, new_pair in zip(tenants, new_key_value_pairs):
    tenant.update(new_pair)

In [61]:
tenants

[{'name': 'Audrey',
  'native_language': 'en',
  'query': 'Who is Wilhelm Weitling?',
  'vector_query': [0.007369487998005005,
   -0.010546107756370203,
   -0.0002366059920899268,
   0.00035501414027413,
   -0.023515060499212228,
   0.019813490993778136,
   -0.02638209419886517,
   0.015533131517677918,
   0.021038373892159318,
   -0.022047894033506436,
   0.021940210792344258,
   0.01634074558184595,
   0.006487841705537974,
   -0.0061345100285955115,
   0.009038559143906022,
   -0.0023370361842392482,
   0.03112010279329178,
   -0.019261620902079984,
   -0.008863575739662628,
   -0.012948762600785403,
   -0.030124043988412504,
   0.003073985216870392,
   -0.004825500296548307,
   0.014119804809908069,
   -0.005700416852103989,
   0.0030891278234128584,
   0.0174714065755328,
   -0.019086638429159163,
   -0.0024228454521331073,
   -0.023124713406612197,
   -0.008325166053109748,
   0.008883766813041823,
   -0.0019601492639538458,
   0.006040288123901178,
   -0.006824348108879489,
   -

### Step 5b. Send Queries!

Now that we have our vectorized queries, we can use our `tenants` dictionary to get each tenants' native language, then send it + the vectorized query together to our index & results back.

In [62]:
# Let's send Audrey's query through to our index
audrey = [t for t in tenants if t.get('name') == 'Audrey'][0]

# Grab Audrey's vectorized query & her native language (which we'll map onto our namespaces)
audrey_query_vector = audrey['vector_query']
audrey_namespace = audrey['native_language']

# Send the query on through!
demo_index.query(vector=audrey_query_vector, top_k=1, include_metadata=True, namespace=audrey_namespace)

# Amazing! We get our chunk of text back that specifically mentions who Wilhelm Weitling is

{'matches': [{'id': 'd4e0bdd2-488b-4155-a2b7-bb6bb7821cf5',
              'metadata': {'chunk_num': 18.0,
                           'text_content': 'of the 19th century such as '
                                           'William Godwin (1756–1836) and '
                                           'Wilhelm Weitling (1808–1871) would '
                                           'contribute to the anarchist '
                                           'doctrines of the next generation '
                                           'but did not use anarchist or '
                                           'anarchism in describing',
                           'title': 'Anarchism',
                           'url': 'https://en.wikipedia.org/wiki/Anarchism'},
              'score': 0.77264607,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': 'en'}

In [63]:
# Let's check out Pierre
pierre = [t for t in tenants if t.get('name') == 'Pierre'][0]

# Grab Pierre's vectorized query & her native language (which we'll map onto our namespaces)
pierre_query_vector = pierre['vector_query']
pierre_namespace = pierre['native_language']

# # Send the query on through!
demo_index.query(vector=pierre_query_vector, top_k=1, include_metadata=True, namespace=pierre_namespace)

# Amazing! We get our chunk of text back that defines what linear algebra is

{'matches': [{'id': '1fc6325b-3ac6-43bf-975c-94f7af71589b',
              'metadata': {'chunk_num': 37.0,
                           'text_content': 'représenter certaines entités '
                                           'physiques comme des déplacements, '
                                           'additionnés entre eux ou encore '
                                           'multipliés par des scalaires '
                                           '(nombres), formant ainsi le '
                                           "premier exemple concret d'espace "
                                           'vectoriel.',
                           'title': 'Algèbre linéaire',
                           'url': 'https://fr.wikipedia.org/wiki/Alg%C3%A8bre%20lin%C3%A9aire'},
              'score': 0.88456255,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': 'fr'}

## BONUS: Filter by Metadata Within a Single Namespace

What if we wanted to filter by metadata...let's say an article's `title`...*within* a specific namespace?

Easy! 

Below we will send a query to our index, to the `'en'` namespace, and filter our results to only be from the Wikipedia article titled `Anarchism`:

In [65]:
# Reminder what our English data looks like
wiki_en.to_pandas().head(3)

,id,url,title,text
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism,Anarchism is a political philosophy and moveme...
1,25,https://en.wikipedia.org/wiki/Autism,Autism,Autism is a neurodevelopmental disorder charac...
2,39,https://en.wikipedia.org/wiki/Albedo,Albedo,Albedo (; ) is the measure of the diffuse refl...


In [66]:
sample_anarchy_query = "What is anarchy?"

In [67]:
vectorized_sample_anarchy_query = vectorize_query(embed, sample_anarchy_query)

In [68]:
targeted_namespace = 'en'

In [69]:
# Send our query through!

demo_index.query(
    vector=vectorized_sample_anarchy_query,
    filter={
        "title": {"$eq": "Anarchism"},
    },
    top_k=3,
    include_metadata=True,
    namespace=targeted_namespace
)

{'matches': [{'id': 'd400facd-2bef-4bdf-8d16-790223037b89',
              'metadata': {'chunk_num': 224.0,
                           'text_content': 'Anarchism and the state',
                           'title': 'Anarchism',
                           'url': 'https://en.wikipedia.org/wiki/Anarchism'},
              'score': 0.87915754,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '7bf128d6-6025-43f1-8e6a-9bbaf80dd99b',
              'metadata': {'chunk_num': 31.0,
                           'text_content': 'society, the rejection of the '
                                           'state apparatus, the belief that '
                                           'human nature allows humans to '
                                           'exist in or progress toward such a '
                                           'non-coercive society, and a '
                                           'suggestion on how to act to pursue

## Awesome!!